In [ ]:
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np

# Remove empty sublists within each list
def clean_array(array):
    return np.array(np.squeeze(ak.Array([[sublist for sublist in inner_list if len(sublist) > 0] for inner_list in array])))


In [27]:
data = ak.from_parquet("hnl_test.parquet")
data.int_probs

<Array [[1.66e-12, 0.953], ..., [1.17e-12, ...]] type='10000 * var * float64'>

In [30]:
sum(data.weights*data.hit_mask_muon0_survival)

2.0789167734311578

In [4]:
import siren
import os
# Now include DIS interaction
cross_section_model = "HNLDISSplines"

xsfiledir = siren.utilities.get_cross_section_model_path(cross_section_model)

# Cross Section Model
target_type = siren.dataclasses.Particle.ParticleType.Nucleon

nutype = "nu"
primary = 14
primary_type = (siren.dataclasses.Particle.ParticleType)(primary)

m4 = "0002000"
Ue4 = 0
Umu4 = 0.01
Utau4 = 0

HNL_DIS_xs = siren.interactions.HNLDISFromSpline(
    os.path.join(xsfiledir, "M_0000000MeV/dsdxdy-%s-N-nc-GRV98lo_patched_central.fits"%nutype),
    os.path.join(xsfiledir, "M_%sMeV/sigma-%s-N-nc-GRV98lo_patched_central.fits"%(m4,nutype)),
    float(m4)*1e-3,
    [Ue4,Umu4,Utau4],
    siren.utilities.Constants.isoscalarMass,
    1,
    [primary_type],
    [target_type],
)
xsfiledir = "/n/holylfs05/LABS/arguelles_delgado_lab/Everyone/pweigel/cross_sections/20241017"

# Cross Section Model
target_type = siren.dataclasses.Particle.ParticleType.Nucleon
nutype = "neutrino"
nuflavor = "muon"
xs_mode = "NC"
minQ2 = 1

DIS_xs = siren.interactions.DISFromSpline(
        os.path.join(xsfiledir, "wcg24b_dsdxdy_%s_%s_%s_isoscalar.fits"%(xs_mode,nuflavor,nutype)),
        os.path.join(xsfiledir, "wcg24b_sigma_%s_%s_%s_isoscalar.fits"%(xs_mode,nuflavor,nutype)),
        1,siren.utilities.Constants.isoscalarMass,minQ2,
        [primary_type],
        [target_type], "cm"
    )

In [11]:

DIS_xs.TotalCrossSection(primary_type,100) # cm^2

1.9537232154384104e-37

In [12]:
HNL_DIS_xs.TotalCrossSection(primary_type,100)# * 1e18 * siren.utilities.Constants.hbarc**2 * 1e4 # cm^2

1.8053350135272425e-41